# Problem Statement

We want to understand what types of topics (subject matter) comprise the content in our documents. We can use **topic modelling** - using statistical methods to discovering the abstract/latent “topics” from a particular corpus.

## Load Data

### BBC News Dataset

We will load a BBC News dataset with documents divided between politics, entertainment, business, sport, and tech.

### BBC Sport Dataset
We will be loading in the BBC Sport news dataset. It is divided between 5 distinct sports topics - football, athletics, cricket, rugby and tennis.

Both datasets are courtesy of *D. Greene and P. Cunningham. "Practical Solutions to the Problem of Diagonal Dominance in Kernel Document Clustering", Proc. ICML 2006.* ([Link](http://mlg.ucd.ie/datasets/bbc.html))


#### Define Function to Load in BBC Datasets

In [ ]:
import pandas as pd
from typing import List, Tuple
def load_bbc_corpus(directory: str, topics: List[str], num_docs: int)-> pd.DataFrame:
    articles: List[Tuple[str, str]] = [(f"../datasets/{directory}/{topic}/{str(i).zfill(3)}.txt", topic) for i in range(1,num_docs + 1) for topic in TOPICS]
    data = []
    for article, topic in articles:
        with open(article, encoding="latin1") as article: # open each sports article
            content = article.read()
            data.append({"topic": topic, "text": content})

    # generate a dataframe
    df = pd.DataFrame(data)
    df.text = df.text.apply(lambda text: text.replace("\n", " "))
    return df

#### Load BBC Sport Corpus

In [ ]:
TOPICS = ["football", "athletics", "cricket", "rugby", "tennis"]
sports_corpus_df = load_bbc_corpus("bbcsport", TOPICS, num_docs=100)
sports_corpus_df.head(5)

#### Load in BBC News Corpus

In [ ]:
documents = []
TOPICS = ["business", "sport", "entertainment", "tech", "politics"]
news_corpus_df = load_bbc_corpus("bbc", TOPICS, num_docs=350)
news_corpus_df.head()

### Technique 1: Non-Negative Matrix Factorization

We can think of our corpus as a two-dimensional table - rows being the documents, and columns being the features (ie. in a count-based vectorizer, each column being a unique token).

In Non-Negative Matrix Factorization, we try to find two matrices `W` and `H`, that contain only nonnegative values and when multiplied together, will reconstruct `X`. 

We need to select a variable `K`, which is the number of components/topics we wish to use.

If we want to model topics for a $N \times M$ matrix `X`, where each value is non-negative, then NMD will produce a $K \times M$ matrix `H` and a $N \times K$ matrix `W`.
![NMF](https://raw.githubusercontent.com/ychennay/dso-560-nlp-text-analytics/main/images/nmf.png)

#### Step 1: Vectorize The Corpus

In [ ]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(2,2),
                             min_df=0.01, max_df=0.4, stop_words="english")

news_vectorizer = TfidfVectorizer(ngram_range=(3,3), min_df=3,
                            max_df=0.4, stop_words="english")

X_sport, sports_terms = vectorizer.fit_transform(sports_corpus_df.text), vectorizer.get_feature_names_out()
X_news, news_terms = news_vectorizer.fit_transform(news_corpus_df.text), news_vectorizer.get_feature_names_out()
sport_tf_idf = pd.DataFrame(X_sport.toarray(), columns=sports_terms)
news_tf_idf = pd.DataFrame(X_news.toarray(), columns=news_terms)
print(f"News TF-IDF: {news_tf_idf.shape}")
print(news_tf_idf.head(5))
print(f"Sports TF-IDF: {sport_tf_idf.shape}")
sport_tf_idf.head(5)

#### Step 2: Fit NMF Model

In [ ]:
nmf = NMF(n_components=5)
W_sport = nmf.fit_transform(X_sport)
H_sport = nmf.components_
print(f"Original shape of X sports is {X_sport.shape}")
print(f"Decomposed W sports matrix is {W_sport.shape}")
print(f"Decomposed H sports matrix is {H_sport.shape}")

In [ ]:
W_news = nmf.fit_transform(X_news)
H_news = nmf.components_
print(f"Original shape of X news is {X_news.shape}")
print(f"Decomposed W news matrix is {W_news.shape}")
print(f"Decomposed H news matrix is {H_news.shape}")

In [ ]:
W_sport

In [ ]:
H_sport

#### Step 3: Report Results For Each Topic

In [ ]:
from typing import List
import numpy as np

def get_top_tf_idf_tokens_for_topic(H: np.array, feature_names: List[str], num_top_tokens: int = 5):
  """
  Uses the H matrix (K components x M original features) to identify for each
  topic the most frequent tokens.
  """
  for topic, vector in enumerate(H):
    print(f"TOPIC {topic}\n")
    total = vector.sum()
    top_scores = vector.argsort()[::-1][:num_top_tokens]
    token_names = list(map(lambda idx: feature_names[idx], top_scores))
    strengths = list(map(lambda idx: vector[idx] / total, top_scores))
    
    for strength, token_name in zip(strengths, token_names):
      print(f"\b{token_name} ({round(strength * 100, 1)}%)\n")
    print(f"=" * 50)

get_top_tf_idf_tokens_for_topic(H_sport, sport_tf_idf.columns.tolist(), 5)
print(f"BBC News Topics:\n\n")
get_top_tf_idf_tokens_for_topic(H_news, news_tf_idf.columns.tolist(), 10)

#### Get the Top Documents For Each Topic

We can also use the `W` matrix to grab top documents per topic (ie. the document that had the greatest percentage of of each topic).

In [ ]:
import numpy as np
def get_top_documents_for_each_topic(W: np.array, documents: List[str], num_docs: int = 5):
    sorted_docs = W.argsort(axis=0)[::-1]
    top_docs = sorted_docs[:num_docs].T
    per_document_totals = W.sum(axis=1)
    for topic, top_documents_for_topic in enumerate(top_docs):
        print(f"Topic {topic}")
        for doc in top_documents_for_topic:
            score = W[doc][topic]
            percent_about_topic = round(score / per_document_totals[doc] * 100, 1)
            print(f"{percent_about_topic}%", documents[doc])
            print("=" * 50)

In [ ]:
get_top_documents_for_each_topic(W_sport, sports_corpus_df.text.tolist())

In [ ]:
get_top_documents_for_each_topic(W_news, news_corpus_df.text.tolist(), num_docs=10)

### Approach 2: LSA (Latent Semantic Analysis)

We can also leverage a dimensionality reduction technique that we've ecnountered before - **Singular Value Decomposition (SVD)** to perform topic modelling.

The following diagram and code snippet is from *Blueprints for Text Analytics Using Python*, Albrecht et al.

Remember that for SVD, we can take our original matrix and decompose it into three matrices.

$$
V = U \times \Sigma \times V^{\star}
$$
We can use these three decomposed matrices for different purposes ([Adel Amadyan](https://adel.ac/singular-value-decomposition-in-computer-vision/#Singular_Value_Decomposition182)):
![SVD Topic Modelling](https://i1.wp.com/adel.ac/wp-content/uploads/2019/02/svd.png?resize=500%2C200) 

The $U$ matrix will provide a signal for what the topic composition of our documents are.

The diagonal elements of the $\Sigma$ matrix can be used to estimate the "strength" of each topic. 

The $V^{star}$ matrix can be used to find the top associated words with each topic.

In [ ]:
from sklearn.decomposition import TruncatedSVD

# we need to select a K (the number of topics)
K = 5

svd = TruncatedSVD(n_components=K)
U = svd.fit_transform(X_news)
V_star = svd.components_

In [ ]:
print(f"U shape is {U.shape}")
get_top_documents_for_each_topic(U, news_corpus_df.text.tolist())

### Approach 3: Latent Dirichlet Allocation

The final approach uses a probablistic sampling method by viewing each document as consisting of mixture of different topics, which are themselves mixtures of different words. 

Each of the mixtures (documents as mixtures of topics, topics as mixtures of words) are modelled using a [Dirichlet distribution](https://en.wikipedia.org/wiki/Dirichlet_distribution).

The algorithm creates initial Dirichlet distributions from each topic and word and tries to recreate the original words used for a document using sampling.

It first attempts to construct the representative words for a topic ([A Beginner's Guide to Latent Dirichlet Allocation, Ria Kulshrestha](https://towardsdatascience.com/latent-dirichlet-allocation-lda-9d1cd064ffa2)):
![https://miro.medium.com/max/1222/1*NjeMT281GMduRYvPIS8IjQ.png](https://miro.medium.com/max/1222/1*NjeMT281GMduRYvPIS8IjQ.png)

The algorithm looks like this:

![LDA](https://miro.medium.com/max/494/1*VTHd8nB_PBsDtd2hd87ybg.png) 

(from [LDA Topic Modeling: An Explanation](https://towardsdatascience.com/lda-topic-modeling-an-explanation-e184c90aadcd))

* $\alpha$ is the per-document topic distributions
* $\phi$ is the word distribution for a given topic
* $\beta$ is the per-topic word distributions
* $\theta$ is the topic distribution for the m-th document
* $Z$ is the topic assigned to the n-th word of the m-th document

We can only observe $W$. β is the table in the above screenshot (each row is a topic, each column is a word). 

We randomly initialize the initial topic distribution, and update iteratively until we converge to a solution or exceed the maximum number of iterations.

The New York Times [highlighed an example of a recommendation system based off of LDA](https://open.blogs.nytimes.com/2015/08/11/building-the-next-new-york-times-recommendation-engine/).

#### Assumptions of LDA
* Bag of words - each document is a bag of words where sequence, part of speech, etc. are not considered. 
* The number of topics is pre-determined and known (or guesstimated).

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=5)
W = lda.fit_transform(X_sport)
get_top_documents_for_each_topic(W, sports_corpus_df.text.tolist(), 10)

### Collaborative Filters

We can also leverage NLP models as part of a collaborative filter in order to generate product recommendations for a given user/customer.

A common approach is constructing an **User-Item** Iteraction Matrix, where there are many sparse elements. We can then iteratively fill compute the User and Item matrices that will minimize the least square error. This class of algorithms is called **Alternating Least Square**.
![https://miro.medium.com/max/1400/1*xMxQL_V9CWeLggrk-Uyzmg.png](https://miro.medium.com/max/1400/1*xMxQL_V9CWeLggrk-Uyzmg.png).

See here for an [example of using collaborative filters in Spark](https://gist.github.com/ychennay/0dc72d5098aa209985feed1a6b4f6b96).

[Himanshu Kriplani, "Alternating Least Square for Implicit Dataset with code"](https://towardsdatascience.com/alternating-least-square-for-implicit-dataset-with-code-8e7999277f4b). 


#### Appendix: Mathematical Theory for Non-Negative Matrix Factorization

Derivations are from [Source Separation Tutorial Mini-Series II: Introduction to Non-Negative Matrix Factorization](https://ccrma.stanford.edu/~njb/teaching/sstutorial/part2.pdf).

We attempt to minimize the divergence $D$, between the original matrix $X$ and the product of the deconstructed $W$ and $H$ matrices:
$$
min(D(V||\hat{V}))
$$
For NMF, this means
$$
min_{W, H >= 0}(D(V||W\times H))
$$
This is read as *we want to select non-negative values for $W$ and $H$ that will minimize $D$*.

There are many functions we can choose to approximate $D$ for examle, **Euclidean Distance**:
$$
D(V||\hat{V}) = \sum_{i,j}{(V_{ij} - \hat{V}_{ij})^{2}}
$$
However, in practice, we commonly select **[Kullback-Leibler Divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence)** to use for the divergence metric:      
$$
D(V||\hat{V}) = \sum_{i,j}{(V_{ij}log(\frac{V_{ij}}{\hat{V}_{ij}}) - V_{ij} + \hat{V}_{ij})}
$$
We can rewrite this as (by substituting $V_{ij}$ with $W\times H$):
$$
D(V||\hat{V}) = \sum_{i,j}{(V_{ij}log(\frac{V_{ij}}{W\times H}) - V_{ij} + W\times H)}
$$

From here, we can use [Jensen's Inequality](https://en.wikipedia.org/wiki/Jensen%27s_inequality) to rewrite this as 
$$
H^{\star}_{kj} = \frac{\sum{V_{ij}\pi_{ijk}}}{\sum{W_{ik}}}
$$
Here, $\pi_{ijk}$ is how much of the component $k$ to assign to the i-th document's j-th feature.
